In [1]:
import onnx
import torch
import torch.onnx

In [77]:
class AddConst(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w = 5
        self.b = torch.nn.Linear(4, 4)
        self.o = torch.tensor([1, 2, 3, 4], dtype=torch.float32)

    def forward(self, x):
        return self.b(x) + self.w + self.o


In [78]:
model = AddConst()
x = torch.randn(3, 4)
torch.onnx.export(model, x, "add_const.onnx", verbose=True)

Exported graph: graph(%onnx::Gemm_0 : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %b.weight : Float(4, 4, strides=[4, 1], requires_grad=1, device=cpu),
      %b.bias : Float(4, strides=[1], requires_grad=1, device=cpu)):
  %/b/Gemm_output_0 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/b/Gemm"](%onnx::Gemm_0, %b.weight, %b.bias), scope: __main__.AddConst::/torch.nn.modules.linear.Linear::b # /Users/benjamintenmann/Library/Caches/pypoetry/virtualenvs/ggml-conversion-fEb3rMr2-py3.11/lib/python3.11/site-packages/torch/nn/modules/linear.py:114:0
  %/Constant_output_0 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={5}, onnx_name="/Constant"](), scope: __main__.AddConst:: # /var/folders/vw/rrzy8ptj6k10mvc6tf4170gw0000gq/T/ipykernel_45008/2052299685.py:9:0
  %/Add_output_0 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = onnx::Add[onnx_name="/Add"](%/b/Gemm_output_0, %/Constant_outp

In [79]:
net = onnx.load("add_const.onnx")
len(net.graph.node)

5

In [30]:
import numpy as np

In [80]:
np.frombuffer(net.graph.node[0].attribute[0].t.raw_data, dtype=np.float32)

array([], dtype=float32)

In [39]:
net.graph.node[0].attribute[0].t.dims

[]

In [45]:
net.graph.node[1].input

['onnx::Add_0', '/Constant_output_0']

In [57]:
net.graph.node[1]

input: "onnx::Add_0"
input: "/Constant_output_0"
output: "2"
name: "/Add"
op_type: "Add"
doc_string: "/var/folders/vw/rrzy8ptj6k10mvc6tf4170gw0000gq/T/ipykernel_45008/2842780031.py(7): forward\n/Users/benjamintenmann/Library/Caches/pypoetry/virtualenvs/ggml-conversion-fEb3rMr2-py3.11/lib/python3.11/site-packages/torch/nn/modules/module.py(1488): _slow_forward\n/Users/benjamintenmann/Library/Caches/pypoetry/virtualenvs/ggml-conversion-fEb3rMr2-py3.11/lib/python3.11/site-packages/torch/nn/modules/module.py(1501): _call_impl\n/Users/benjamintenmann/Library/Caches/pypoetry/virtualenvs/ggml-conversion-fEb3rMr2-py3.11/lib/python3.11/site-packages/torch/jit/_trace.py(118): wrapper\n/Users/benjamintenmann/Library/Caches/pypoetry/virtualenvs/ggml-conversion-fEb3rMr2-py3.11/lib/python3.11/site-packages/torch/jit/_trace.py(127): forward\n/Users/benjamintenmann/Library/Caches/pypoetry/virtualenvs/ggml-conversion-fEb3rMr2-py3.11/lib/python3.11/site-packages/torch/nn/modules/module.py(1501): _call_i

In [75]:
net.graph.initializer

google._upb._message.RepeatedCompositeContainer

In [71]:
net.graph.input[0].type.tensor_type

elem_type: 1
shape {
  dim {
    dim_value: 3
  }
  dim {
    dim_value: 4
  }
}

In [89]:
net.graph.node[-4].attribute[0].t.dims

[]

In [86]:
net.graph.node[-2].attribute[0].t.dims

[4]